In [6]:
import pandas as pd
import seaborn as sb
import csv
from sympy import FiniteSet

import pysam
import os
import math
import random
import numpy as np

from pathlib import Path
from random import seed
from random import randint

In [ ]:
path=

Vamos a mapear los reads verdaderos de maiz a los genomas de la base de datos interna de referencia de _Clavibacter_ (BDIRC) 
0. Crear link simbolicos
1. Indexar los genomas de BDIRC con bowtie
2. Mapear con bowtie a esos genomas
3. Si queremos visualizar los reads podemos usar tablet, y necesitamos samtools para indexar el genoma y ver los reads alineados.
https://www.metagenomics.wiki/tools/bowtie2

In [2]:
%%bash 
source ~/anaconda3/etc/profile.d/conda.sh
conda activate metagenomics

A continuación creamos indices con para cada uno de los genomas refencia, para ello utilizamos el comando `bowtie2-build`. Los respectivos archivos de salida los almacenamos en la carpeta `index-bowtie`. Adicionalmente, creamos el archivo `list-genomes.txt` que contiene los nombre de cada archivo de salida del `bowtie2-build`.

In [3]:
%%bash 
source ~/anaconda3/etc/profile.d/conda.sh
conda activate metagenomics
ls genomas-clavi/clavi-genomes/*fna | while read line; 
do 
    id=$(echo $line| rev| cut -d'_' -f1,2,3 |rev |cut -d'.' -f1);
    bowtie2-build $line genomas-clavi/clavi-genomes/index-bowtie/$id;
    echo "$id" >>list-genomes.txt
done

#Por ahora solo corre en bash





Settings:
  Output files: "genomas-clavi/clavi-genomes/index-bowtie/subsp_capsici_1101.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  genomas-clavi/clavi-genomes/Clavibacter_michiganensis_subsp_capsici_1101.fna
Reading reference sizes
  Time reading reference sizes: 00:00:00
Total time for call to driver() for forward index: 00:00:00
Settings:
  Output files: "genomas-clavi/clavi-genomes/index-bowtie/subsp_capsici_1106.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: 

Building a SMALL index
Could not open index file for writing: "genomas-clavi/clavi-genomes/index-bowtie/subsp_capsici_1101.3.bt2"
Please make sure the directory exists and that permissions allow writing by
Bowtie.
Error: Encountered internal Bowtie 2 exception (#1)
Command: /opt/anaconda3/envs/metagenomics/bin/bowtie2-build-s --wrapper basic-0 genomas-clavi/clavi-genomes/Clavibacter_michiganensis_subsp_capsici_1101.fna genomas-clavi/clavi-genomes/index-bowtie/subsp_capsici_1101 
Deleting "genomas-clavi/clavi-genomes/index-bowtie/subsp_capsici_1101.3.bt2" file written during aborted indexing attempt.
Deleting "genomas-clavi/clavi-genomes/index-bowtie/subsp_capsici_1101.4.bt2" file written during aborted indexing attempt.
Building a SMALL index
Could not open index file for writing: "genomas-clavi/clavi-genomes/index-bowtie/subsp_capsici_1106.3.bt2"
Please make sure the directory exists and that permissions allow writing by
Bowtie.
Error: Encountered internal Bowtie 2 exception (#1)
Comm

Lo siguiente es mapear los reads a los genomas de referencia, utilizamos `bowtie2` y obtenemos arhivos de salidas *.sam* que guardaremos en la carpeta `output-bowtie` 

#reference=subsp_capsici_1101  ; usar para extraer las lineas de archivo sed '1q;d' index-genomes.txt
for infile in ex-reads-clavi/fastq/fastq/capsicum/*-clav-1.fq
 do 
base=$(basename ${infile} -clav-1.fq)
dir=$(dirname ${infile} )
bowtie2 -x genomas-clavi/clavi-genomes/index-bowtie/subsp_capsici_1101 -1 ${dir}/${base}-clav-1.fq -2 ${dir}/${base}-clav-2.fq  --no-unal -p 12 -S genomas-clavi/clavi-genomes/out-bowtie/${base}.sam
done

In [4]:
%%bash 
source ~/anaconda3/etc/profile.d/conda.sh
conda activate metagenomics
bash 01-mapear-reads-a-referencia.sh insidiosus_CFBP_2404

(ERR): "genomas-clavi/clavi-genomes/index-bowtie/insidiosus_CFBP_2404" does not exist or is not a Bowtie 2 index
Exiting now ...
(ERR): "genomas-clavi/clavi-genomes/index-bowtie/insidiosus_CFBP_2404" does not exist or is not a Bowtie 2 index
Exiting now ...
(ERR): "genomas-clavi/clavi-genomes/index-bowtie/insidiosus_CFBP_2404" does not exist or is not a Bowtie 2 index
Exiting now ...
(ERR): "genomas-clavi/clavi-genomes/index-bowtie/insidiosus_CFBP_2404" does not exist or is not a Bowtie 2 index
Exiting now ...
(ERR): "genomas-clavi/clavi-genomes/index-bowtie/insidiosus_CFBP_2404" does not exist or is not a Bowtie 2 index
Exiting now ...
(ERR): "genomas-clavi/clavi-genomes/index-bowtie/insidiosus_CFBP_2404" does not exist or is not a Bowtie 2 index
Exiting now ...
(ERR): "genomas-clavi/clavi-genomes/index-bowtie/insidiosus_CFBP_2404" does not exist or is not a Bowtie 2 index
Exiting now ...
(ERR): "genomas-clavi/clavi-genomes/index-bowtie/insidiosus_CFBP_2404" does not exist or is not a

Ahora corremos el scrip `01-mapear-reads-a-referencia.sh` por cada una de la lista de genomas que se encuentra en `index-genomes.txt´ 

In [ ]:
cat index-genomes.txt | while read line;
do
    bash 01-mapear-reads-a-referencia.sh $line
done

## Prueba 1: Utilizar solo 3 genomas
Ahora procedor hacer lo anterior son con 3 genomas, 1 de Clavibacter Michiganensis Michiganensis (cmm), y 2 de Michagenesis capsisi. 
Primero cremos los index



In [5]:
%%bash
source ~/anaconda3/etc/profile.d/conda.sh
conda activate metagenomics
ls genomas-clavi/clavi-genomes/genomas-prueba1/*fna | while read line; 
do 
    id=$(echo $line| rev| cut -d'_' -f1,2,3 |rev |cut -d'.' -f1);
    bowtie2-build $line genomas-clavi/clavi-genomes/genomas-prueba1/index-bowtie/$id;
    echo "$id" >>list-genomasrefencia.txt
done

Settings:
  Output files: "genomas-clavi/clavi-genomes/genomas-prueba1/index-bowtie/subsp_capsici_1101.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  genomas-clavi/clavi-genomes/genomas-prueba1/Clavibacter_michiganensis_subsp_capsici_1101.fna-cortado.fna
Reading reference sizes
  Time reading reference sizes: 00:00:00
Total time for call to driver() for forward index: 00:00:00
Settings:
  Output files: "genomas-clavi/clavi-genomes/genomas-prueba1/index-bowtie/subsp_capsici_1106.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)

Building a SMALL index
Could not open index file for writing: "genomas-clavi/clavi-genomes/genomas-prueba1/index-bowtie/subsp_capsici_1101.3.bt2"
Please make sure the directory exists and that permissions allow writing by
Bowtie.
Error: Encountered internal Bowtie 2 exception (#1)
Command: /opt/anaconda3/envs/metagenomics/bin/bowtie2-build-s --wrapper basic-0 genomas-clavi/clavi-genomes/genomas-prueba1/Clavibacter_michiganensis_subsp_capsici_1101.fna-cortado.fna genomas-clavi/clavi-genomes/genomas-prueba1/index-bowtie/subsp_capsici_1101 
Deleting "genomas-clavi/clavi-genomes/genomas-prueba1/index-bowtie/subsp_capsici_1101.3.bt2" file written during aborted indexing attempt.
Deleting "genomas-clavi/clavi-genomes/genomas-prueba1/index-bowtie/subsp_capsici_1101.4.bt2" file written during aborted indexing attempt.
Building a SMALL index
Could not open index file for writing: "genomas-clavi/clavi-genomes/genomas-prueba1/index-bowtie/subsp_capsici_1106.3.bt2"
Please make sure the directory e

Ahora, hacemos el mapeo de los read simulados a cada genoma de refencia previamente indexado


In [6]:
%%bash
source ~/anaconda3/etc/profile.d/conda.sh
conda activate metagenomics

cat list-genomasrefencia.txt | while read line;
do
bowtie2 -x output-tda/prueba-3genomas/index-bowtie/$line -U reads-simulados100M.fastq --no-unal -p 12 -S output-tda/prueba-3genomas/out-bowtie/Simulados-${line}.sam
done

700000 reads; of these:
  700000 (100.00%) were unpaired; of these:
    232352 (33.19%) aligned 0 times
    460838 (65.83%) aligned exactly 1 time
    6810 (0.97%) aligned >1 times
66.81% overall alignment rate
700000 reads; of these:
  700000 (100.00%) were unpaired; of these:
    231193 (33.03%) aligned 0 times
    462956 (66.14%) aligned exactly 1 time
    5851 (0.84%) aligned >1 times
66.97% overall alignment rate
700000 reads; of these:
  700000 (100.00%) were unpaired; of these:
    254665 (36.38%) aligned 0 times
    421192 (60.17%) aligned exactly 1 time
    24143 (3.45%) aligned >1 times
63.62% overall alignment rate
700000 reads; of these:
  700000 (100.00%) were unpaired; of these:
    232352 (33.19%) aligned 0 times
    460838 (65.83%) aligned exactly 1 time
    6810 (0.97%) aligned >1 times
66.81% overall alignment rate
700000 reads; of these:
  700000 (100.00%) were unpaired; of these:
    231193 (33.03%) aligned 0 times
    462956 (66.14%) aligned exactly 1 time
    5851

### Crear archivos .bam para visualizar y extraer información del match

In [7]:
%%bash
for infile in output-tda/prueba-3genomas/out-bowtie/*.sam
do 
	base=$(basename ${infile} .sam)
	samtools view -b -S ${infile} -o output-tda/prueba-3genomas/bam/${base}.bam
done

Crea un archivo *.txt con la salida mostrando el identificador y el read que identifico en el genoma referencia.

In [13]:
%%bash
for infile in output-tda/prueba-3genomas/bam/*.bam
do 
	base=$(basename ${infile} .bam)
	samtools sort ${infile} -o output-tda/prueba-3genomas/sorted/${base}_sorted.bam
	samtools view ${infile} > output-tda/prueba-3genomas/bam/${base}.txt
done

Obtener las columnas que nos interesan de los archivos .sam

In [10]:
%%bash
cd ~/GIT/clavibacter/scripts-tda/
ls output-tda/prueba-3genomas/bam/*.txt | while read line
do
    base=$(basename ${line} .txt)
    cut -f 1-6 $line > output-tda/prueba-3genomas/tablas/${base}_extrac.txt
done


Importar tablas de mapeos


In [2]:
ruta_tablas = '/home/betterlab/GIT/clavibacter/scripts-tda/output-tda/prueba-3genomas/tablas/'
contenido = os.listdir(ruta_tablas)
genomes = []
df=[]
for i in contenido:
    dftt = pd.read_csv(ruta_tablas + i, sep='\t', header=None) #importar las tablas para cada genoma
    dftt['Organims']=i[16:-11] #agregar el nombre del organismo
    df.append(dftt) #agregar al dataframe
    genomes.append(i[16:-11])
    
df=pd.concat(df)
df.columns= ['Qname','Flag', 'Rname', 'Pos','MapQ','CIGAR','Organims'] #cambiar nombre a las columnas del dataframe
df 

,Qname,Flag,Rname,Pos,MapQ,CIGAR,Organims
0,NZ_CP048049.1_0,16,NZ_CP048048.1,111591,42,150M,capsici_1106
1,NC_009478.1_0,0,NZ_CP048047.1,2691532,42,150M,capsici_1106
2,NC_009479.1_0,0,NZ_CP048048.1,81328,42,150M,capsici_1106
3,NZ_CP048048.1_0,0,NZ_CP048047.1,407360,42,150M,capsici_1106
4,NZ_CP048050.1_1,0,NZ_CP048048.1,44206,42,150M,capsici_1106
...,...,...,...,...,...,...,...
445330,NZ_CP048049.1_99997,0,NC_009480.1,243049,0,42M1I3M1D104M,michi_contigs
445331,NZ_CP048050.1_99997,0,NC_009480.1,2942610,42,150M,michi_contigs
445332,NC_009480.1_99997,0,NC_009479.1,69488,42,150M,michi_contigs
445333,NC_009478.1_99997,0,NC_009480.1,1808419,0,150M,michi_contigs


In [3]:
df.to_csv('MapeoReads-organismos.csv')

En 'df2' agrupamos por nombre del read y por organimos.
En 'df3'agrupamos por read y obtenemos a cuantos organismos mapea


In [74]:
df2=df.groupby(['Qname','Organims'])['Qname'].size()
df2
df3=df.groupby(['Qname'])['Qname'].count().reset_index(name='count')
df2


Qname                Organims     
NC_009478.1_0        capsici_1101     1
                     capsici_1106     1
                     michi_contigs    1
NC_009478.1_1        michi_contigs    1
NC_009478.1_10       capsici_1101     1
                                     ..
NZ_CP048050.1_99998  capsici_1101     1
                     capsici_1106     1
NZ_CP048050.1_99999  capsici_1101     1
                     capsici_1106     1
                     michi_contigs    1
Name: Qname, Length: 1381790, dtype: int64

In [75]:
df2.reset_index()

ValueError: cannot insert Qname, already exists

In [21]:
X=df['Organims'].unique().tolist()
Xset(X)

Definimos la funcion potencia

In [22]:
def powerset(fullset):
  listsub = list(fullset)
  subsets = []
  for i in range(2**len(listsub)):
    subset = []
    for k in range(len(listsub)):            
      if i & 1<<k:
        subset.append(listsub[k])
    subsets.append(subset)        
  return subsets

In [23]:
powerset(X)

[[],
 ['michi_contigs'],
 ['capsici_1101'],
 ['michi_contigs', 'capsici_1101'],
 ['capsici_1106'],
 ['michi_contigs', 'capsici_1106'],
 ['capsici_1101', 'capsici_1106'],
 ['michi_contigs', 'capsici_1101', 'capsici_1106']]

In [28]:
df.loc[df.loc[:,'Organims']=='michi_contigs']

,Qname,Flag,Rname,Pos,MapQ,CIGAR,Organims
0,NZ_CP048050.1_0,0,NC_009479.1,45254,42,150M,michi_contigs
1,NC_009480.1_0,0,NC_009478.1,10211,42,150M,michi_contigs
2,NC_009478.1_0,0,NC_009480.1,2947543,23,150M,michi_contigs
3,NZ_CP048047.1_0,0,NC_009479.1,13579,42,150M,michi_contigs
4,NZ_CP048049.1_1,0,NC_009480.1,29946,42,150M,michi_contigs
...,...,...,...,...,...,...,...
445330,NZ_CP048049.1_99997,0,NC_009480.1,243049,0,42M1I3M1D104M,michi_contigs
445331,NZ_CP048050.1_99997,0,NC_009480.1,2942610,42,150M,michi_contigs
445332,NC_009480.1_99997,0,NC_009479.1,69488,42,150M,michi_contigs
445333,NC_009478.1_99997,0,NC_009480.1,1808419,0,150M,michi_contigs


AttributeError: 'Series' object has no attribute 'Qname'